## Setup

Downloads the FHIR Validator from https://github.com/hapifhir/org.hl7.fhir.core/releases/latest/download/validator_cli.jar

For more details on the FHIR Validator see [Using the FHIR Validator](https://confluence.hl7.org/spaces/FHIR/pages/35718580/Using+the+FHIR+Validator)

Downloads the NW Genomics validation package

In [11]:
from tqdm import tqdm
import requests

print("Downloading the FHIR Validator...")

url = "https://github.com/hapifhir/org.hl7.fhir.core/releases/latest/download/validator_cli.jar"
response = requests.get(url, stream=True)

with open("validator_cli.jar", "wb") as handle:
    for data in tqdm(response.iter_content(chunk_size=1024), unit="kB"):
        handle.write(data)

print("Downloading the North West Genomics validation package...")
url = "https://nw-gmsa.github.io/package.tgz"
response = requests.get(url, stream=True)

with open("package.tgz", "wb") as handle:
    for data in tqdm(response.iter_content(chunk_size=1024), unit="kB"):
        handle.write(data)


181476kB [00:06, 30153.89kB/s]


4597kB [00:00, 6560.97kB/s]


Now run a validation using a FHIR Message Bundle.

The example is an R01 Report, so we also need to help the validator by specifying the profile of the focus resource which in this case is a DiagnosticReport which should adhere to https://nw-gmsa.github.io/StructureDefinition-DiagnosticReport.html (profile url https://fhir.nwgenomics.nhs.uk/StructureDefinition/DiagnosticReport)

A default output is returned from the validator.

Using command line this is:

>  java -jar validator_cli.jar Output/FHIR/R01/LRI-GeneVariant-3.txt.json -version 4.0.1 -ig package.tgz -bundle DiagnosticReport:0 https://fhir.nwgenomics.nhs.uk/StructureDefinition/DiagnosticReport -tx n/a

In [12]:
import subprocess

filename = "LRI-GeneVariant-3.txt.json"
folder = 'FHIR/R01/'

subprocess.run(["java", "-jar", "validator_cli.jar", "Output/"+folder + filename , "-version", "4.0.1", "-ig", "package.tgz", "-bundle", "DiagnosticReport:0", "https://fhir.nwgenomics.nhs.uk/StructureDefinition/DiagnosticReport", "-tx", "n/a"])

FHIR Validation tool Version 6.8.0 (Git# 16897818c932). Built 2026-02-03T21:15:40.739Z (12 days old)
  Java:   21.0.7 from /Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home on x86_64 (64bit). 8192MB available
  Paths:  Current = /Users/kevinmayfield/github/MFT/Testing, Package Cache = /Users/kevinmayfield/.fhir/packages
  Params: Output/FHIR/R01/LRI-GeneVariant-3.txt.json -version 4.0.1 -ig package.tgz -bundle DiagnosticReport:0 https://fhir.nwgenomics.nhs.uk/StructureDefinition/DiagnosticReport -tx n/a
  Locale: United Kingdom/GB
  Jurisdiction: United Kingdom of Great Britain and Northern Ireland
Loading
  Loading FHIR v4.0.1 from hl7.fhir.r4.core#4.0.1
  Load hl7.terminology.r4#6.2.0 - 4288 resources (00:10.773)
  Load hl7.fhir.uv.extensions.r4#5.2.0 - 759 resources (00:03.002)
  Loaded FHIR - 8265 resources (00:00.000)
  Terminology server null - Version n/a: No Terminology Server (00:00.000)
  Load hl7.fhir.uv.extensions.r5#5.2.0 - 759 resources (00:01.314)
  Load hl7.ter

CompletedProcess(args=['java', '-jar', 'validator_cli.jar', 'Output/FHIR/R01/LRI-GeneVariant-3.txt.json', '-version', '4.0.1', '-ig', 'package.tgz', '-bundle', 'DiagnosticReport:0', 'https://fhir.nwgenomics.nhs.uk/StructureDefinition/DiagnosticReport', '-tx', 'n/a'], returncode=0)

The output can contain a mix of issues which may be difficult to understand. If we get a large number of issues, we may want to prioritise `error` first and write a test report. So we need to output the results in a machine readable format



In [13]:
import subprocess

subprocess.run(["java", "-jar", "validator_cli.jar", "Output/"+folder + filename, "-version", "4.0.1", "-ig", "package.tgz", "-bundle", "DiagnosticReport:0", "https://fhir.nwgenomics.nhs.uk/StructureDefinition/DiagnosticReport", "-tx", "n/a", "-output", "Results/"+folder + filename + "-OperationOutcome.json", "-output-style","json"])

FHIR Validation tool Version 6.8.0 (Git# 16897818c932). Built 2026-02-03T21:15:40.739Z (12 days old)
  Java:   21.0.7 from /Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home on x86_64 (64bit). 8192MB available
  Paths:  Current = /Users/kevinmayfield/github/MFT/Testing, Package Cache = /Users/kevinmayfield/.fhir/packages
  Params: Output/FHIR/R01/LRI-GeneVariant-3.txt.json -version 4.0.1 -ig package.tgz -bundle DiagnosticReport:0 https://fhir.nwgenomics.nhs.uk/StructureDefinition/DiagnosticReport -tx n/a -output Results/FHIR/R01/LRI-GeneVariant-3.txt.json-OperationOutcome.json -output-style json
  Locale: United Kingdom/GB
  Jurisdiction: United Kingdom of Great Britain and Northern Ireland
Loading
  Loading FHIR v4.0.1 from hl7.fhir.r4.core#4.0.1
  Load hl7.terminology.r4#6.2.0 - 4288 resources (00:10.860)
  Load hl7.fhir.uv.extensions.r4#5.2.0 - 759 resources (00:03.252)
  Loaded FHIR - 8265 resources (00:00.000)
  Terminology server null - Version n/a: No Terminology Server 

CompletedProcess(args=['java', '-jar', 'validator_cli.jar', 'Output/FHIR/R01/LRI-GeneVariant-3.txt.json', '-version', '4.0.1', '-ig', 'package.tgz', '-bundle', 'DiagnosticReport:0', 'https://fhir.nwgenomics.nhs.uk/StructureDefinition/DiagnosticReport', '-tx', 'n/a', '-output', 'Results/FHIR/R01/LRI-GeneVariant-3.txt.json-OperationOutcome.json', '-output-style', 'json'], returncode=0)

In [14]:
import json
import pandas as pd

def details(item):
    return item['text']

with open( "Results/"+folder + filename + "-OperationOutcome.json", 'rb') as f:
    s = f.read().decode('utf-8')
    operationOutcome = json.loads(s)

df = pd.DataFrame(operationOutcome['issue'])

df['details'] = df['details'].apply(details)
df.drop(columns=['extension'], inplace=True)
df.sort_values(by=['severity'], inplace=True)


df = df[~df['details'].str.contains('ValueSet/mimetypes')]
df = df[~df['details'].str.contains('failed: dom-6')]
df = df[~df['details'].str.contains('bcp:13')]

df

,severity,code,details,expression
67,error,invalid,This element does not match any known slice de...,[Bundle.entry[5].resource/*Observation/null*/....
64,error,invalid,This element does not match any known slice de...,[Bundle.entry[5].resource/*Observation/null*/....
92,error,invalid,This element does not match any known slice de...,[Bundle.entry[6].resource/*Observation/null*/....
113,error,structure,Unable to find a profile match for urn:uuid:5f...,[Bundle.entry[8].resource.result[2]]
109,error,processing,Slicing cannot be evaluated: Could not match d...,[Bundle.entry[8].resource.category[0]]
...,...,...,...,...
41,warning,not-found,A definition for CodeSystem 'http://varnomen.h...,[Bundle.entry[5].resource/*Observation/null*/....
42,warning,not-found,A definition for CodeSystem 'http://www.ncbi.n...,[Bundle.entry[5].resource/*Observation/null*/....
43,warning,not-found,A definition for CodeSystem 'Chrom-Loc' could ...,[Bundle.entry[5].resource/*Observation/null*/....
65,warning,not-found,A definition for CodeSystem 'http://www.genena...,[Bundle.entry[5].resource/*Observation/null*/....


In [15]:
import jinja2
import weasyprint
from datetime import date

html_string = df.to_html()

html = (
    jinja2.Environment(loader=jinja2.FileSystemLoader(searchpath=""))
    .get_template(name="Results/report_template.html")
    .render(
        date=date.today().strftime("%d, %b %Y"),
        table=html_string
    )
)

with open("Results/"+folder + filename + "-report.html", "w") as out_html_file_handle:
    out_html_file_handle.write(html)

with open("Results/"+folder + filename + "-report.html", "r") as html_file_handle:
    pdf = weasyprint.HTML(html_file_handle).write_pdf()


open("Results/"+folder + filename + '-report.pdf', 'wb').write(pdf)



31547